<a href="https://www.kaggle.com/code/akitomosato/disaster-bert?scriptVersionId=99479380" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


https://htomblog.com/python-bert

In [2]:
!pip install transformers
!pip install nlp
!pip install datasets # コーパス読み込みのため

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.7 MB/s eta 0:00:00


In [3]:
%config Completer.use_jedi = False # タブ補完

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data_train = pd.read_csv('../input/nlp-getting-started/train.csv')
data_test = pd.read_csv('../input/nlp-getting-started/test.csv')
data_submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

For each ID in the test set, you must predict 1 if the tweet is describing a real disaster, and 0 otherwise.

[pandasで欠損値nanじゃないデータを抽出する方法](https://qiita.com/ponsuke0531/items/7df79686a660e6b025cb)

In [6]:
display(data_train)
display(data_test)
display(data_train.dropna(subset=['keyword'])) # keywordがNaNでないものを確認

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


In [7]:
data_submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


# 主要な統計手法をまとめて確認

In [8]:
data_train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [9]:
data_train.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

# データの概要をざっと確認

In [10]:
# import pandas_profiling
# data_train.profile_report()

# 特徴量エンジニアリング

In [11]:
# 教師データとテストデータ、双方にエンジニアリングを行うため、一旦ひとつに結合
data_all = pd.concat([data_train, data_test],sort=False)
data_all

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,NaN
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,NaN
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN


In [12]:
# 各特徴量の欠損値の数を確認
data_all.isnull().sum()

id             0
keyword       87
location    3638
text           0
target      3263
dtype: int64

In [13]:
data_all.dtypes

id            int64
keyword      object
location     object
text         object
target      float64
dtype: object

## とりあえず、テキストのみで学習してみる

In [14]:
drop_columns = ['id','keyword','location']
data_all.drop(drop_columns, axis=1,inplace=True) # axis=1は列

In [15]:
# 欠損値がないか確認
data_all.isnull().sum()

text         0
target    3263
dtype: int64

In [16]:
# 結合したデータを、再度教師データとテストデータを分離
data_train = data_all[:len(data_train)]
data_test = data_all[len(data_train):]

In [17]:
data_train.isnull().sum()

text      0
target    0
dtype: int64

In [18]:
data_train['target'] = data_train['target'].astype('int')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
data_train.dtypes

text      object
target     int64
dtype: object

In [20]:
data_test.isnull().sum()

text         0
target    3263
dtype: int64

In [21]:
# アルゴリズムに投入するため、特徴量と目的変数を分離
# y_data_train = data_train['target']
# X_data_train = data_train.drop(['target'],axis=1)
# X_data_test = data_test.drop(['target'],axis=1)

# 訓練データを学習用と検証用に分割

In [22]:
import csv
from sklearn.model_selection import train_test_split

In [23]:
print(data_train.columns)
data_train.columns = [col.replace('target','label') for col in data_train.columns]

Index(['text', 'target'], dtype='object')


In [24]:
tweet_train, tweet_test = train_test_split(data_train, random_state=111) # 訓練用とテスト用に分割 defalut 25%がテストデータ
tweet_path = "./"

In [25]:
print(tweet_train.columns)
# tweet_train.drop('', axis=1,inplace=True) # axis=1は列
display(tweet_train)

Index(['text', 'label'], dtype='object')


,text,label
2248,Businesses are deluged with invoices. Make you...,0
2042,I wish I could get Victoria's Secret on front....,0
5300,Families to sue over Legionnaires: More than 4...,1
6837,Hollywood Movie About Trapped Miners Released ...,1
1897,@CBSBigBrother ouch Clelli....you could almost...,0
...,...,...
4182,Choking Hazard Prompts Recall Of Kraft Cheese ...,1
4820,If abortion is murder then blowjobs are cannib...,1
2004,@swb1192 if the NDA is written to damage your ...,0
7020,Global precipitation measurement satellite cap...,1


In [26]:
# trainerではColumnは不要
tweet_train.to_csv(tweet_path+"tweet_train.csv", index=False,header=False)
tweet_test.to_csv(tweet_path+"tweet_test.csv", index=False,header=False)

In [27]:
# with open(tweet_path+"tweet_train.csv","w") as f:
#   writer = csv.writer(f)
#   writer.writerows(tweet_train)

# with open(tweet_path+"tweet_test.csv","w") as f:
#   writer = csv.writer(f)
#   writer.writerows(tweet_test)

In [28]:
# # 学習データ、検証データに 8:2 の割合で分割する。
# # train_size = int(0.8 * len(data_train))
# # indices = np.arange(len(data_train))

# # train_dataset = torch.utils.data.Subset(data_train, indices[:train_size])
# # val_dataset = torch.utils.data.Subset(data_train, indices[train_size:])


# print(f"full: {len(data_train)} -> train: {len(train_dataset)}, test: {len(val_dataset)}")

In [29]:
display(pd.read_csv(tweet_path+"tweet_train.csv"))
display(pd.read_csv(tweet_path+"tweet_test.csv"))
# display(val_dataset)

,Businesses are deluged with invoices. Make yours stand out with colour or shape and it's likely to ris. togthe top of the pay' pile.,0
0,I wish I could get Victoria's Secret on front....,0
1,Families to sue over Legionnaires: More than 4...,1
2,Hollywood Movie About Trapped Miners Released ...,1
3,@CBSBigBrother ouch Clelli....you could almost...,0
4,Versions of KS where if a character was /every...,0
...,...,...
5703,Choking Hazard Prompts Recall Of Kraft Cheese ...,1
5704,If abortion is murder then blowjobs are cannib...,1
5705,@swb1192 if the NDA is written to damage your ...,0
5706,Global precipitation measurement satellite cap...,1


,Do you feel engulfed with low self-image? Take the quiz: http://t.co/ykVsttvDWo http://t.co/IFQQpUr99X,0
0,That moment when ur about to win a mini uhc an...,0
1,@IAN_Hellfire I got it for the mistake but bos...,0
2,Humboldt Cty Sheriff's Office has issued an ev...,1
3,A spider has legit just run across my chest. T...,0
4,./.....hmm 12000 Nigerian refugees repatriated...,1
...,...,...
1898,Wreckage 'Conclusively Confirmed' as From MH37...,1
1899,NOTHING YOU MIDGET I WILL DEMOLISH YOU SHOW SO...,0
1900,Dad bought a DVD that looks like a science doc...,1
1901,I'm on top of the hill and I can see a fire in...,1


# モデル作成

In [30]:
from transformers import BertForSequenceClassification, BertTokenizer

sc_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',  num_labels=2)
sc_model.cuda()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [31]:
from datasets import load_dataset

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

# news_path = "/content/drive/MyDrive/Colab Notebooks/livedoor news courpus/"

train_data = load_dataset("csv", data_files=tweet_path+"tweet_train.csv", column_names=["text", "label"], split="train")
display(train_data)
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
display(train_data)
train_data.set_format("torch", columns=["input_ids", "label"])
display(train_data)

test_data = load_dataset("csv", data_files=tweet_path+"tweet_test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-15e47eb1621220e0/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 5709
})

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5709
})

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5709
})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ffce6086cc184447/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

# 評価用の関数

In [32]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels,preds)
    return {
        "accuracy":acc,
    }

# Trainerの設定
Trainerクラス、及びTrainingArgumentsクラスを使用して、訓練を行うTrainerの設定を行います。

In [33]:
!mkdir ./logs
!ls

__notebook__.ipynb  logs  tweet_test.csv  tweet_train.csv


In [34]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./",
    num_train_epochs = 4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps = 500, #学習係数が0からこのステップで上昇
    weight_decay = 0.01, #重みの減衰率
    logging_dir = "./logs",
    # evaluate_during_training=True, # バグ発生、下で代替
    evaluation_strategy='steps',
    report_to=None,
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data,
)

In [35]:
display(trainer)

# モデルの訓練
設定に基づきファインチューニング

In [36]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [37]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5709
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2856
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Accuracy
500,0.520200,0.473617,0.791492
1000,0.458300,0.487968,0.816702
1500,0.391100,0.602839,0.838235
2000,0.306900,0.564972,0.832983
2500,0.247000,0.737969,0.809874


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 32
Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 32
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
The following columns in the 

TrainOutput(global_step=2856, training_loss=0.36196898879791173, metrics={'train_runtime': 384.6175, 'train_samples_per_second': 59.373, 'train_steps_per_second': 7.426, 'total_flos': 985753791126720.0, 'train_loss': 0.36196898879791173, 'epoch': 4.0})

# モデルの評価

In [38]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1904
  Batch size = 32


{'eval_loss': 0.7342012524604797,
 'eval_accuracy': 0.8130252100840336,
 'eval_runtime': 5.8533,
 'eval_samples_per_second': 325.284,
 'eval_steps_per_second': 10.251,
 'epoch': 4.0}

# TensorBoardによる結果の評価

In [39]:
%load_ext tensorboard
%tensorboard --logdir logs

In [40]:
%reload_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 374), started 0:00:00 ago. (Use '!kill 374' to kill it.)